In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S18'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 5 #From 3 to 5 as in paper
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 10000
Total Tokens Found = 130023
Total Tokens Output = 127367
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 10000
Total Tokens= 127367
Unique Tokens= 20063
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  5
Maximum Frequency of Error Token =  3
References Processed= 10000
Sorted Token Size = 20063
Clean Token Size = 7551
*Stop Replacements here
References Processed =  10000
Total Replacement Pairs = 2321
Tokens Read = 127367
Tokens Changed =  2717
References Changed = 2470


In [2]:
status="done"

In [3]:
print(status)

done


In [4]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [5]:
import numpy as np

Create a function of the parameters that returns the F score

In [6]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [7]:
entropy_F1(mu=0.5,muIncr=0.05,beta=33,sigma=449,epsilon=31.8,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 10202
NewMatrix Iterateblocks Total Time = 11.025781663134694


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9561
NewMatrix Iterateblocks Total Time = 11.452074885368347


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8200
NewMatrix Iterateblocks Total Time = 9.04804590344429


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 5203
NewMatrix Iterateblocks Total Time = 5.531879342161119


>>>>>>>>>>>>
Starting Iteration mu= 0.7000000000000002
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!

0.72553812

Create functions for variable transformation.

In [8]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [9]:
inv_softplus(1)

0.541324854612918

In [10]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [11]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [12]:
import os, sys

In [14]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [15]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [16]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [16]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


0.9669684330973831

Bounds for experiment

In [94]:
beta_levels=(1,350,700)
sigma_levels=(1,350,700)
mu_levels=(0.5,0.75,0.99)
muIncr_levels=(0.01,0.1,0.2)
epsilon_levels=(1,350,700)

In [137]:
optimizer=None

In [138]:
pbounds = {'beta_sp': (inv_softplus(beta_levels[0]), inv_softplus(beta_levels[2])), 
           'sigma_sp': (inv_softplus(sigma_levels[0]), inv_softplus(sigma_levels[2])),
           'mu_logit': (logit(mu_levels[0]), logit(mu_levels[2])),
           'muIncr_logit': (logit(muIncr_levels[0]), logit(muIncr_levels[2])),
           'epsilon_sp': (inv_softplus(epsilon_levels[0]), inv_softplus(epsilon_levels[2]))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [139]:
from bayes_opt import SequentialDomainReductionTransformer

In [140]:
bounds_transformer = SequentialDomainReductionTransformer()

In [141]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1#,
    #bounds_transformer=bounds_transformer
)

In [142]:
import pyDOE2

In [143]:
levels = [3, 3, 3, 3, 3]

In [144]:
gsd_design=pyDOE2.gsd(levels=levels,reduction=5)

In [145]:
for exp in gsd_design:
    optimizer.probe(
        params={"beta_sp": inv_softplus(beta_levels[exp[0]]), 
                "sigma_sp": inv_softplus(sigma_levels[exp[1]]),
                "mu_logit":logit(mu_levels[exp[2]]),
                #'epsilonIterate_logit': logit(0),
                "muIncr_logit":logit(muIncr_levels[exp[3]]),
                "epsilon_sp":inv_softplus(epsilon_levels[exp[4]])},
        lazy=True,
    )

In [146]:
optimizer.maximize(
    init_points=0,
    n_iter=100-gsd_design.shape[0],
    acq="ucb",
    kappa=2
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.50|Mu+:0.01|Beta:1.00|Sigma:1.00|Epsilon:1.00|EpIter:0.00
|  1        | -124.3    |  0.5413   |  0.5413   | -4.595    |  0.0      |  0.5413   |
Mu:0.50|Mu+:0.10|Beta:1.00|Sigma:1.00|Epsilon:350.00|EpIter:0.00
|  2        | -124.3    |  0.5413   |  350.0    | -2.197    |  0.0      |  0.5413   |
Mu:0.50|Mu+:0.20|Beta:1.00|Sigma:1.00|Epsilon:700.00|EpIter:0.00
|  3        | -124.3    |  0.5413   |  700.0    | -1.386    |  0.0      |  0.5413   |
Mu:0.75|Mu+:0.01|Beta:1.00|Sigma:1.00|Epsilon:350.00|EpIter:0.00
|  4        | -124.3    |  0.5413   |  350.0    | -4.595    |  1.099    |  0.5413   |
Mu:0.75|Mu+:0.10|Beta:1.00|Sigma:1.00|Epsilon:700.00|EpIter:0.00
|  5        | -124.3    |  0.5413   |  700.0    | -2.197    |  1.099    |  0.5413   |
Mu:0.99|Mu+:0.01|Beta:1.00|Sigma:1.00|Epsilon:700.00|EpIter:0.00
|  6      

|  39       | -6.76     |  700.0    |  0.5413   | -2.197    |  0.0      |  700.0    |
Mu:0.50|Mu+:0.20|Beta:700.00|Sigma:700.00|Epsilon:350.00|EpIter:0.00
|  40       | -6.074    |  700.0    |  350.0    | -1.386    |  0.0      |  700.0    |
Mu:0.75|Mu+:0.01|Beta:700.00|Sigma:700.00|Epsilon:1.00|EpIter:0.00
|  41       | -4.781    |  700.0    |  0.5413   | -4.595    |  1.099    |  700.0    |
Mu:0.75|Mu+:0.10|Beta:700.00|Sigma:700.00|Epsilon:350.00|EpIter:0.00
|  42       |  0.156    |  700.0    |  350.0    | -2.197    |  1.099    |  700.0    |
Mu:0.75|Mu+:0.20|Beta:700.00|Sigma:700.00|Epsilon:700.00|EpIter:0.00
|  43       |  0.156    |  700.0    |  700.0    | -1.386    |  1.099    |  700.0    |
Mu:0.99|Mu+:0.01|Beta:700.00|Sigma:700.00|Epsilon:350.00|EpIter:0.00
|  44       | -3.51     |  700.0    |  350.0    | -4.595    |  4.595    |  700.0    |
Mu:0.99|Mu+:0.10|Beta:700.00|Sigma:700.00|Epsilon:700.00|EpIter:0.00
|  45       | -3.51     |  700.0    |  700.0    | -2.197    |  4.595    

|  78       | -2.121    |  343.7    |  696.4    | -4.428    |  3.118    |  353.6    |
Mu:0.97|Mu+:0.17|Beta:344.00|Sigma:341.00|Epsilon:685.49|EpIter:0.00
|  79       | -2.4      |  345.0    |  685.5    | -1.556    |  3.317    |  341.1    |
Mu:0.86|Mu+:0.10|Beta:693.00|Sigma:341.00|Epsilon:343.45|EpIter:0.00
|  80       | -0.7721   |  693.4    |  343.4    | -2.179    |  1.797    |  341.9    |
Mu:0.93|Mu+:0.17|Beta:332.00|Sigma:350.00|Epsilon:690.68|EpIter:0.00
|  81       | -1.369    |  332.8    |  690.7    | -1.573    |  2.562    |  350.5    |
Mu:0.65|Mu+:0.12|Beta:697.00|Sigma:350.00|Epsilon:349.38|EpIter:0.00
|  82       | -0.7736   |  698.0    |  349.4    | -2.028    |  0.6214   |  350.4    |
Mu:0.91|Mu+:0.03|Beta:351.00|Sigma:347.00|Epsilon:693.03|EpIter:0.00
|  83       | -1.162    |  351.7    |  693.0    | -3.465    |  2.362    |  347.9    |
Mu:0.94|Mu+:0.04|Beta:359.00|Sigma:344.00|Epsilon:358.18|EpIter:0.00
|  84       | -1.625    |  359.0    |  358.2    | -3.226    |  2.792  

In [135]:
optimizer.max

{'target': 0.4309728831511782,
 'params': {'beta_sp': 350.04984627254953,
  'epsilon_sp': 32.81832802950333,
  'muIncr_logit': -1.9270464035913897,
  'mu_logit': 0.9596395754580471,
  'sigma_sp': 699.3973744268105}}

In [136]:
inv_logit(optimizer.max["target"])

0.60610596

In [51]:
params=optimizer.max["params"]

In [52]:
softplus(params["beta_sp"])

29.851689931200212

In [53]:
softplus(params["sigma_sp"])

453.49601863602436

In [54]:
inv_logit(params["mu_logit"])

0.5

In [55]:
inv_logit(params["muIncr_logit"])

0.051004766758926935

In [45]:
inv_logit(params["epsilonIterate_logit"])

KeyError: 'epsilonIterate_logit'

In [56]:
softplus(params["epsilon_sp"])

29.18200686707413

In [57]:
entropy_F1(mu=0.5,
           muIncr=0.051004766758926935,
           beta=29,
           sigma=453,
           epsilon=29.18200686707413,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 29.18200686707413
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9900
NewMatrix Iterateblocks Total Time = 10.462520653381944


>>>>>>>>>>>>
Starting Iteration mu= 0.551004766758927
Starting Iteration epsilon= 29.18200686707413
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9174
NewMatrix Iterateblocks Total Time = 10.022749735973775


>>>>>>>>>>>>
Starting Iteration mu= 0.6020095335178539
Starting Iteration epsilon= 29.18200686707413
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 7954
NewMatrix Iterateblocks Total Time = 8.64361101295799


>>>>>>>>>>>>
Starting Iteration mu= 0.6530143002767809
Starting Iteration epsilon= 29.18200686707413
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 5754
NewMatrix Iterateblocks Total Time = 6.570374641567469


>>>>>>>>>>>>
Starting Iteration mu= 0.70401906

0.73010211